In [1]:
import ogr, os
from math import ceil
import shapefile as shp
from shapely.geometry import Point

pointfeat = '/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/ChinaVIIRS.shp' #starting points
# pointfeat = '/Users/itclunie/Desktop/ECON/find industry/speedTest/chinaTest.shp'

gridPath = "/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/countryGrid.shp" #folder to dump results
# gridPath = '/Users/itclunie/Desktop/ECON/find industry/speedTest/countryGrid.shp'

gridHeight = .02 #.015 = 1.5km
gridWidth = .02

In [2]:
def makeGrid(outputGridfn,xmin,xmax,ymin,ymax,gridHeight,gridWidth):
    # convert sys.argv to float
    xmin = float(xmin)
    xmax = float(xmax)
    ymin = float(ymin)
    ymax = float(ymax)
    gridWidth = float(gridWidth)
    gridHeight = float(gridHeight)

    # get rows
    rows = ceil((ymax-ymin)/gridHeight)
    # get columns
    cols = ceil((xmax-xmin)/gridWidth)

    # start grid cell envelope
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin + gridWidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax-gridHeight

    # create output file
    outDriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputGridfn):
        os.remove(outputGridfn)
        
    outDataSource = outDriver.CreateDataSource(outputGridfn)
    outLayer = outDataSource.CreateLayer(outputGridfn,geom_type=ogr.wkbPolygon )
    featureDefn = outLayer.GetLayerDefn()

    # create grid cells
    countcols = 0
    while countcols < cols:
        countcols += 1

        # reset envelope for rows
        ringYtop = ringYtopOrigin
        ringYbottom =ringYbottomOrigin
        countrows = 0

        while countrows < rows:
            countrows += 1
            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            # add new geom to layer
            outFeature = ogr.Feature(featureDefn)
            outFeature.SetGeometry(poly)
            outLayer.CreateFeature(outFeature)
            outFeature.Destroy

            # new envelope for next poly
            ringYtop = ringYtop - gridHeight
            ringYbottom = ringYbottom - gridHeight

        # new envelope for next poly
        ringXleftOrigin = ringXleftOrigin + gridWidth
        ringXrightOrigin = ringXrightOrigin + gridWidth

    # Close DataSources
    outDataSource.Destroy()

In [3]:
pointfeatExtent = shp.Reader(pointfeat)

xmin = pointfeatExtent.bbox[0]
xmax = pointfeatExtent.bbox[2]
ymin = pointfeatExtent.bbox[1]
ymax = pointfeatExtent.bbox[3]

makeGrid(gridPath,xmin,xmax,ymin,ymax,gridHeight,gridWidth)